In [1]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
from __future__ import division
import pickle
import random

### Get list of movies

Find 300 top grossing movies per year for 2004 - 2014

In [2]:
url_base = 'http://www.imdb.com/search/title?at=0&sort=boxoffice_gross_us&start=' 
url_feat = '&title_type=feature&year='

In [179]:
titles = {}
counts = [i for i in range(1,300,50)]

In [180]:
year = 2000

for count in counts:
    start = count
    url = url_base + str(start) + url_feat + str(year)
    print "Getting data " + str(start) + " for " + str(year)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)

    lst = soup.find_all(attrs={"href" : re.compile("/title/")})
    for link in lst:
        if len(link.text) > 0 and link.text != 'X':
            titles[link.attrs['href']] = link.text

Getting data 1 for 2000
Getting data 51 for 2000
Getting data 101 for 2000
Getting data 151 for 2000
Getting data 201 for 2000
Getting data 251 for 2000


In [181]:
sorted(titles.values())[:10]

[u'102 Dalmatians',
 u'28 Days',
 u'3 Strikes',
 u'A Time for Drunken Horses',
 u'Aberdeen',
 u'About Adam',
 u'All the Pretty Horses',
 u'Almost Famous',
 u'American Psycho',
 u'Amores Perros']

In [182]:
len(titles.items())

300

In [183]:
len(movies_dct.items())

4124

In [60]:
# movies_dct = {}

In [71]:
def add_movie_to_data(soup2, href):
    title = soup2.find(itemprop = "name").text
    metacritic = str(soup2.find(attrs={"href" : re.compile("criticreviews?")}).text.strip())
    info = soup2.find(class_ = 'infobar')
    meta = info.find_all('meta')
    
    genres = []
    genres2 = info.find_all(itemprop='genre')
    for genre in genres2:
        genres.append(str(genre.text))
    duration = info.find(itemprop='duration').attrs['datetime']
    opening = soup2.find(text=re.compile("Opening Weekend:")).next.strip()
    studio = soup2.find(attrs={"href" : re.compile("/company/")}).text.strip()
    
    try:
        rating = info.find(itemprop='contentRating')['content']
    except TypeError:
        rating = 'UNKNOWN'
    
    try:
        release = info.find(itemprop='datePublished')['content']
    except TypeError:
        release = 0
    
    # get correct gross
    url_bus = url_page + 'business?ref_=tt_dt_bus'
    response3 = requests.get(url_bus)
    page3 = response3.text
    soup3 = BeautifulSoup(page3)
    
    try:
        gross = soup3.find(text=re.compile("Gross")).next.strip().split()[0]
    except AttributeError:
        gross = soup2.find(text=re.compile("Gross:")).next.strip()
    
    movies_dct[href] = {'title': title,
                         'metacritic': metacritic,
                         'rating': rating,
                         'genres': genres,
                         'release': release,
                         'duration': duration,
                         'studio': studio,
                         'gross': gross,
                         'opening': opening
                        }
    if len(movies_dct.items()) % 50 == 0:
        with open('my_data_partial.pkl', 'w') as picklefile:
            pickle.dump(movies_dct.items(), picklefile)
            print "dumped " + str(len(movies_dct.items()))
    return movies_dct

In [184]:
for href, title in titles.items():
    url_page = 'http://www.imdb.com/' + href

    try:
        response2 = requests.get(url_page)
        page2 = response2.text
        soup2 = BeautifulSoup(page2)
        add_movie_to_data(soup2, href)
        sleep(0.4)

    except AttributeError:
        print title + ' did not load'
        sleep(0.2)

dumped 4150
Raju Chacha did not load
Chal Mere Bhai did not load
dumped 4200
Refugee did not load
dumped 4250
One Week did not load
Phir Bhi Dil Hai Hindustani did not load
Khauff did not load
dumped 4300
dumped 4350
Kahin Pyaar Na Ho Jaaye did not load
Hadh Kar Di Aapne did not load
Mela did not load
dumped 4400


In [191]:
len(movies_dct.items())

4415

In [192]:
(len(movies_dct.items())-4124)/len(titles.items())

0.97

In [193]:
movies_dct.items()[:4]

[('/title/tt0388838/',
  {'duration': 'PT90M',
   'genres': ['Drama', 'Music'],
   'gross': u'$136,007',
   'metacritic': '75/100',
   'opening': u'$5,700        \n\n      (Hong Kong)',
   'rating': 'R',
   'release': '2004-09-01',
   'studio': u'Rectangle Productions',
   'title': u'Clean'}),
 ('/title/tt0368794/',
  {'duration': 'PT135M',
   'genres': ['Biography', 'Drama', 'Music'],
   'gross': u'$4,017,609',
   'metacritic': '73/100',
   'opening': u'$730,819        \n\n      (USA)',
   'rating': 'R',
   'release': '2007-12-07',
   'studio': u'Killer Films',
   'title': u"I'm Not There."}),
 ('/title/tt0203540/',
  {'duration': 'PT91M',
   'genres': ['Comedy', 'Crime', 'Romance'],
   'gross': u'$1,439,287',
   'metacritic': '46/100',
   'opening': u'$65,584        \n\n      (USA)',
   'rating': 'R',
   'release': '2001-09-14',
   'studio': u'Boneyard Entertainment',
   'title': u'Greenfingers'}),
 ('/title/tt1433905/',
  {'duration': 'PT144M',
   'genres': ['Action', 'Comedy', 'Dra

In [188]:
with open('my_data_2000_2014.pkl', 'w') as picklefile:
    pickle.dump(movies_dct.items(), picklefile)

In [194]:
count = 0
for i in range(len(movies_dct.items())):
    if (movies_dct.items()[i][1]['metacritic'] == 'Metacritic Reviews'):
        count += 1
        
print count

317


In [190]:
count / len(movies_dct.items())

0.07180067950169876

### MISC JUNK

In [44]:
url_page = 'http://www.imdb.com/title/tt3093546/'

response2 = requests.get(url_page)
page2 = response2.text
soup2 = BeautifulSoup(page2)

In [ ]:
#soup2.prettify()

In [45]:
soup2.find(itemprop = "name").text

u'Listen Up Philip'

In [46]:
str(soup2.find(attrs={"href" : re.compile("criticreviews?")}).text.strip())

'76/100'

In [47]:
info = soup2.find(class_ = 'infobar')

In [48]:
len(info)

13

In [49]:
info.find(itemprop='contentRating')['content']

TypeError: 'NoneType' object has no attribute '__getitem__'

In [50]:
meta = info.find_all('meta')

In [51]:
meta

[<meta content="2014-10-21" itemprop="datePublished"/>]

In [52]:
info.find(itemprop='duration').attrs['datetime']

'PT108M'

In [53]:
str(soup2.find(text=re.compile("Opening Weekend:")).next.strip())

'$23,284        \n\n      (USA)'

In [54]:
str(soup2.find(attrs={"href" : re.compile("/company/")}).text.strip())

'Sailor Bear'

In [55]:
str(soup2.find(text=re.compile("Release Date:")).next.strip())

'21 October 2014 (USA)'

In [56]:
url_bus = url_page + 'business?ref_=tt_dt_bus'
response3 = requests.get(url_bus)
page3 = response3.text
soup3 = BeautifulSoup(page3)

In [57]:
url_bus

'http://www.imdb.com/title/tt3093546/business?ref_=tt_dt_bus'

In [58]:
soup3.find(text=re.compile("Gross")).next.strip().split()[0]

AttributeError: 'NoneType' object has no attribute 'next'